In [ ]:
using ITensors
using Plots
using LinearAlgebra
plot_font = "Computer Modern";

# TEBD for real-time evolution of XXZ spin model

The Hamiltonian of the XXZ spin-1/2 chain is given by: $H = J \sum_{j=1}^{N-1} \left(S_j^x S_{j+1}^x + S_j^y S_{j+1}^y + \Delta S_j^z S_{j+1}^z\right) + B \sum_{j=1}^N (-1)^j S_j^z$
where $J = 1$, $N = 24$, and $S_j^{\alpha}$ are spin-1/2 operators.



Setup parameters

In [ ]:
N = 24   # Number of sites  

J = 1.0   # Hopping
Δ = 1.00   # entopy
B = 1.0   # Bfield

T = 8.0  # Final time
δt = 0.01   # Time step
time = 0.0:δt:T   # Time vector
tbigstep = 20   # Calculate expectation values each tbigstep times

num_expvals = Int(((length(time)-1)/tbigstep)) + 1; # Number of times expectation values will be calculated. The ±1 is to account correctly for t = 0  
cutoff = 1E-8;   # Truncation allowed per step
χ = 100; # Maximum bond dimension

s = siteinds("S=1/2",N);

### a) Define an initial MPS in which the first half of the lattice has spins pointing up, and the second half has spins pointing down (3 points).
In this step, we define an initial MPS where:
- The first half of the lattice ($1 \leq j \leq 12$) has spins pointing up ($S^z = +1$).
- The second half ($13 \leq j \leq 24$) has spins pointing down ($S^z = -1$).

In [ ]:
function InitialState(s,N)


   state = [n<=N/2 ? "Up" : "Dn" for n=1:N];
   @show state
   ψ0 = MPS(s,state);
   
   return ψ0;
end

### b) Implement a TEBD code for simulating the time evolution of the initial MPS under the Hamiltonian of Eq. (1), up to a final time T = 8.0. Select adequate values for the time step δt and truncation parameters (20 points).

In [ ]:
Norm = zeros(num_expvals,1); # Norm of evolved state
Popul = zeros(num_expvals,N); # Number of bosons per site
Coherences = zeros(num_expvals,N,N)+1im*zeros(num_expvals,N,N); # Coherences between all sites, can be complex
SvN = zeros(num_expvals,N-1); # von Neumann entanglement entropy
Time_expvals = zeros(num_expvals,1); # Time of expectation values

Function to calculate expectation values

In [ ]:
function ExpVals!(ψ,N,Popul,Coherences,Norm,timeval) # Here, timeval is the position in the arrays of expectation values where info will be stored 

   # The functions expect and correlation_matrix normalize internally the expectation values
   Popul[timeval,:] = real(expect(ψ, "Sz")); # Population of each site
   Coherences[timeval,:,:] = correlation_matrix(ψ,"Sx","Sx"); # Single particle density matrix
   Norm[timeval,1] = norm(ψ); # Norm of the state
    
end;

Function to calculate von Neumann entanglement entropy

In [ ]:
function Entropy!(state, N, SvN, timeval)

    SvN_time = zeros(N-1)
    
    for b=1:N-1
        orthogonalize!(state, b)
        if b == 1
            U,S,V = svd(state[b], (siteind(state, b))) # There is no link to the left
        else
            U,S,V = svd(state[b], (linkind(state, b-1), siteind(state, b)))
        end
        for n=1:dim(S, 1)
            p = S[n,n]^2
            SvN_time[b] -= p * log(p)
        end
    end

    SvN[timeval,:] = SvN_time
    
    #return SvN
    
end

Define sequence of local evolution propagators
$H = J \sum_{j=1}^{N-1} \left(S_j^x S_{j+1}^x + S_j^y S_{j+1}^y + \Delta S_j^z S_{j+1}^z\right) + B \sum_{j=1}^N (-1)^j S_j^z$

In [ ]:
function staircase_gates(s,N,δt,J,Δ,B)

    gates = ITensor[] # Initialize network of two-site gates

    # Sweep over pairs of sites
    for j in 1:(N - 1)
        
        s1 = s[j]
        s2 = s[j + 1]
    
        # Define factors for single-site operators
        fs1 = 0.5;
        fs2 = 0.5;
        if(j == 1)
            fs1 = 1;
        elseif(j == N-1)
            fs2 = 1;
        end
        staggered=isodd(j) ? -1 : 1
        #staggered=1
        # Define two-site Hamiltonian
        hj = J*op("Sx",s1)*op("Sx",s2)+J*op("Sy",s1)*op("Sy",s2)+J*Δ*op("Sz",s1)*op("Sz",s2); # Hopping
        hj = hj + B*staggered*(4/3)*fs1*op("Sz",s1)*op("S2",s2) ; # Interaction of left site
        hj = hj - B*staggered*(4/3)*fs2*op("S2",s1)*op("Sz",s2) ; # Interaction of right site
        
        # Create local gate and include in total propagator
        Gj = exp(-1im*0.5*δt*hj)
        push!(gates, Gj)
    end
    
    # Include gates in reverse order too (N,N-1),(N-1,N-2),...
    append!(gates, reverse(gates));
    
    return gates
end;    

----------- Main code of time evolution -----------

Define initial state

In [ ]:
s = siteinds("S=1/2",N);

In [ ]:
ψ0 = InitialState(s,N);

Calculate initial expectation values and entropy

In [ ]:
ExpVals!(ψ0, N, Popul, Coherences, Norm, 1);

#@show Popul[1,:];
#@show Coherences

In [ ]:
Entropy!(ψ0, N, SvN, 1);

@show SvN[1,:];

Create gate staircase for time evolution

In [ ]:
gates = staircase_gates(s,N,δt,J,Δ,B);

Perform time evolution

In [ ]:
ψ = ψ0

count_expvals = 1;

for t in 1:length(time)-1

    ψ = apply(gates, ψ; cutoff=cutoff, maxdim=χ)
    #normalize!(ψ)
    
    if(mod(t,tbigstep)== 0)
       
        println("Calculating expectation values for $(t) number of steps")
        count_expvals = count_expvals + 1;

        Time_expvals[count_expvals] = t*δt;
            
        # The state is not normalized, just the expectation values in the function
        ExpVals!(ψ, N, Popul, Coherences, Norm, count_expvals);
        Entropy!(ψ, N, SvN, count_expvals);                

    end
    
end;

Plot results

In [ ]:
@show size(Popul')

In [ ]:
j_array = 1:N
heatmap(Time_expvals[:,1], j_array, Popul', xlabel = "Time", ylabel = "j", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)
#title!('Sz')

In [ ]:
plot(Time_expvals,Popul[:,Int(0.5*N)], xlabel = "Time", ylabel = "Sz at site N/2", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)

In [ ]:
jmin1_array = 1:N-1
heatmap(Time_expvals[:,1], jmin1_array, SvN', xlabel = "Time", ylabel = "j", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)

In [ ]:
plot(Time_expvals,SvN[:,Int(0.5*N)], xlabel = "Time", ylabel = "Entanglement at bond N/2", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)

Simulation finished!!!

### c) Obtain the magnetization along z at every site of the spin chain for times t = 0, 2, 4, 6, 8, and for the following combinations of parameters Δ and B. Plot and save the results of each combination of Δ and B on a separate figure (2 points each figure).

i. Δ = 0, B = 0.   
ii. Δ = 0, B = 2.   
iii. Δ = 2, B = 0.  

### d) Obtain the von Neumann entanglement entropy at every bond of the spin chain for times t = 0, 2, 4, 6, 8, and for the same combinations of parameters Δ and B of c). Plot and save the results of each combination of Δ and B on a separate figure (2 points each figure).

In [ ]:
J = 1.0   # Hopping
Δ = 0.00   # entopy
B = 0.0   # Bfield

T = 8.0  # Final time
δt = 0.01   # Time step
time = 0.0:δt:T   # Time vector
tbigstep = 200 ;  # Calculate expectation values each tbigstep times

num_expvals = Int(((length(time)-1)/tbigstep)) + 1; # Number of times expectation values will be calculated. The ±1 is to account correctly for t = 0  
cutoff = 1E-8;   # Truncation allowed per step
χ = 100; # Maximum bond dimension

In [ ]:
Norm = zeros(num_expvals,1); # Norm of evolved state
Popul = zeros(num_expvals,N); # Number of bosons per site
Coherences = zeros(num_expvals,N,N)+1im*zeros(num_expvals,N,N); # Coherences between all sites, can be complex
SvN = zeros(num_expvals,N-1); # von Neumann entanglement entropy
Time_expvals = zeros(num_expvals,1); # Time of expectation values

In [ ]:
gates = staircase_gates(s,N,δt,J,Δ,B);

In [ ]:
ψ = ψ0

count_expvals = 1;

for t in 1:length(time)-1

    ψ = apply(gates, ψ; cutoff=cutoff, maxdim=χ)
    #normalize!(ψ)
    
    if(mod(t,tbigstep)== 0)
       
        println("Calculating expectation values for $(t) number of steps")
        count_expvals = count_expvals + 1;

        Time_expvals[count_expvals] = t*δt;
            
        # The state is not normalized, just the expectation values in the function
        ExpVals!(ψ, N, Popul, Coherences, Norm, count_expvals);
        Entropy!(ψ, N, SvN, count_expvals);                

    end
    
end;

In [ ]:
Popul0= zeros(N);
Norm0 = norm(ψ0);
Popul0[:] = real(expect(ψ0, "Sz"))
plot(Popul0,label="t = 0")
plot!(Popul'[:,2],label="t = 2")
plot!(Popul'[:,3],label="t = 4")
plot!(Popul'[:,4],label="t = 6")
plot!(Popul'[:,5],label="t = 8")
title!("Δ = $Δ, B=$B,  Sz magnetization")

In [ ]:
#SvN0 = zeros(N-1);
#SvN0[:] = real(expect(ψ0, "Sz"))/Norm0^2

plot( SvN'[:,1],label="t = 0")
plot!(SvN'[:,2],label="t = 2")
plot!(SvN'[:,3],label="t = 4")
plot!(SvN'[:,4],label="t = 6")
plot!(SvN'[:,5],label="t = 8")
title!("Δ = $Δ, B=$B,  von Neumann entanglement entropy")

In [ ]:

Δ = 0.00 ;  # entopy
B = 2.0 ;  # Bfield

In [ ]:
gates = staircase_gates(s,N,δt,J,Δ,B);

In [ ]:
ψ = ψ0

count_expvals = 1;

for t in 1:length(time)-1

    ψ = apply(gates, ψ; cutoff=cutoff, maxdim=χ)
    #normalize!(ψ)
    
    if(mod(t,tbigstep)== 0)
       
        println("Calculating expectation values for $(t) number of steps")
        count_expvals = count_expvals + 1;

        Time_expvals[count_expvals] = t*δt;
            
        # The state is not normalized, just the expectation values in the function
        ExpVals!(ψ, N, Popul, Coherences, Norm, count_expvals);
        Entropy!(ψ, N, SvN, count_expvals);                

    end
    
end;

In [ ]:
Sz0= zeros(N);
Norm0 = norm(ψ0);
Sz0[:] = real(expect(ψ0, "Sz"))/Norm0^2
plot(Sz0,label="t = 0")
plot!(Popul'[:,2],label="t = 2")
plot!(Popul'[:,3],label="t = 4")
plot!(Popul'[:,4],label="t = 6")
plot!(Popul'[:,5],label="t = 8")
title!("Δ = $Δ, B=$B,  Sz magnetization")

In [ ]:
plot( SvN'[:,1],label="t = 0")
plot!(SvN'[:,2],label="t = 2")
plot!(SvN'[:,3],label="t = 4")
plot!(SvN'[:,4],label="t = 6")
plot!(SvN'[:,5],label="t = 8")
title!("Δ = $Δ, B=$B,  von Neumann entanglement entropy")

In [ ]:

Δ = 2.00 ;  # entopy
B = 0.0 ;  # Bfield
gates = staircase_gates(s,N,δt,J,Δ,B);
ψ = ψ0

count_expvals = 1;

for t in 1:length(time)-1

    ψ = apply(gates, ψ; cutoff=cutoff, maxdim=χ)
    #normalize!(ψ)
    
    if(mod(t,tbigstep)== 0)
       
        println("Calculating expectation values for $(t) number of steps")
        count_expvals = count_expvals + 1;

        Time_expvals[count_expvals] = t*δt;
            
        # The state is not normalized, just the expectation values in the function
        ExpVals!(ψ, N, Popul, Coherences, Norm, count_expvals);
        Entropy!(ψ, N, SvN, count_expvals);                

    end
    
end;

In [ ]:
Sz0= zeros(N);
Norm0 = norm(ψ0);
Sz0[:] = real(expect(ψ0, "Sz"))/Norm0^2
plot(Sz0,label="t = 0")
plot!(Popul'[:,2],label="t = 2")
plot!(Popul'[:,3],label="t = 4")
plot!(Popul'[:,4],label="t = 6")
plot!(Popul'[:,5],label="t = 8")
title!("Δ = $Δ, B=$B,  Sz magnetization")

In [ ]:
plot( SvN'[:,1],label="t = 0")
plot!(SvN'[:,2],label="t = 2")
plot!(SvN'[:,3],label="t = 4")
plot!(SvN'[:,4],label="t = 6")
plot!(SvN'[:,5],label="t = 8")
title!("Δ = $Δ, B=$B,  von Neumann entanglement entropy")

# 2. TDVP for ground state of spin model with next-nearest neighbor interactions (25 points)

### a) Using imaginary time evolution with TDVP, obtain the ground state |ψ0⟩ and ground state energy E0 of the Hamiltonian for N = 30, J1 = 1, J2 = 0.5, and h = 2. Feel free to choose t

In [ ]:
using HDF5
using ITensors
using LinearAlgebra
using Plots
using ITensorMPS
using Observers
using LaTeXStrings
plot_font = "Computer Modern";

In [ ]:
N = 30 # System size
J1 = 2; # Hopping parameters
J2 = 0.2;
h=2;
#PBC = 0; # Periodic boundary conditions

# Define N S=1/2 spin indices
sites = siteinds("S=1/2",N);

In [ ]:

T = 0.1  # Final time
δt = 0.001   # Time step
time = 0.0:δt:T   # Time vector
tbigstep = 10   # Calculate expectation values each tbigstep times
num_expvals = Int(((length(time)-1)/tbigstep)) + 1; # Number of times expectation values will be calculated. The ±1 is to account correctly for t = 0  
cutoff = 1E-10;   # Truncation allowed per step
χ = 300; # Maximum bond dimension

In [ ]:
E0 = zeros(num_expvals); # Norm of evolved state
#Popul = zeros(num_expvals,N); # Number of bosons per site
#Coherences = zeros(num_expvals,N,N)+1im*zeros(num_expvals,N,N); # Coherences between all sites, can be complex
Time_expvals = zeros(num_expvals,1); # Time of expectation values

In [ ]:
function Hamiltonian(N,sites,J1,J2,h)

    # Initiate construction
    ops = OpSum()

    # Hopping terms
    for j=1:N-1
        ops += J1,"Sz",j,"Sz",j+1
        ops += J1,"Sz",j,"Sz",j+1    
    end
    for j=1:N-2
        ops += J2,"Sz",j,"Sz",j+2
        ops += J2,"Sz",j,"Sz",j+2    
    end
    for j=1:N
        ops += -h,"Sx",j
    end

    # PBC if required
   

    H = MPO(ops,sites);
    
    return H;
    
end;

In [ ]:
function InitialState(sites,N)
    
    state = [isodd(n) ? "+" : "+" for n=1:N];
    ψi = MPS(sites,state);
    
    return ψi;
end

In [ ]:
function GroundState(H,ψi)

    # Do many sweeps of DMRG, gradually increasing the maximum MPS bond dimension
    nsweeps = 100;
    maxdim = 100;
    cutoff = 1E-12;

    # Run the DMRG algorithm for ground state   
    energy0,ψ0 = dmrg(H,ψi; nsweeps, maxdim, cutoff, outputlevel=1);

    @show energy0

    # Orthonormalize state. Leave site 1 as orthogonalization center
    orthogonalize!(ψ0,1)

    return energy0, ψ0;
    
end

In [ ]:
H = Hamiltonian(N,sites,J1,J2,h);
ψ0 = InitialState(sites,N);

In [ ]:
ψ = ψ0

count_expvals = 1;

for t in 1:length(time)-1
    
    ψ = tdvp(H, -im * δt, ψ; time_step = -im * δt, cutoff = cutoff, maxdim = χ, outputlevel=0, normalize=true);
    
    normalize!(ψ)
    
    if(mod(t,tbigstep)== 0)
       
        println("Calculating expectation values for $(t) number of steps")
        count_expvals = count_expvals + 1;

        Time_expvals[count_expvals] = t*δt;
        #normalize!(ψ)
        # The state is not normalized, just the expectation values in the function
        E0[count_expvals ] = real(inner(ψ',H,ψ));
    end
    
end;

### b) Plot the evolution of the energy as a function of the imaginary time τ (5 points).

In [ ]:
plot(Time_expvals,E0, xlabel = "Time", ylabel = "E0", xtickfontsize = 15, ytickfontsize = 15, xguidefontsize = 15, yguidefontsize = 15, colorbar_tickfontsize = 15, c = :plasma, fontfamily=plot_font)

In [ ]:
H1=real(inner(ψ',H,ψ));
@show H1

In [ ]:
H2 = real(inner(H,ψ,H,ψ));
H1=real(inner(ψ',H,ψ));

var = H2-H1^2;
@show var

In [ ]:
# Define spin Hamiltonian
H = Hamiltonian(N,sites,J1,J2,h);
ψi = InitialState(sites,N);
# Calculate ground state
E0,ψ0 = GroundState(H,ψi);

In [ ]:
ψ = ψ0

count_expvals = 1;
N_f=100;
ni=1;
while var>1e-6 && ni<N_f
    
    
    ψ = tdvp(H, -im * δt, ψ; time_step = -im * δt, cutoff = cutoff, maxdim = χ, outputlevel=0, normalize=true);
    
    normalize!(ψ)
    H2 = real(inner(H,ψ,H,ψ));
    H1=real(inner(ψ',H,ψ));
    var = H2-H1^2;

    if(mod(ni,tbigstep)== 0)
       
        println("Calculating expectation values for $(ni) number of steps and var=$(var)")
        count_expvals = count_expvals + 1;

       # Time_expvals[count_expvals] = ni*δt;
        #normalize!(ψ)
        # The state is not normalized, just the expectation values in the function
        #E0[count_expvals ] = real(inner(ψ',H,ψ));
    end
    
    ni=ni+1;
    
end;
@show H1

### b) Calculate and plot the ground state expectation values ⟨Sxj ⟩ for all sites j (2 points).

In [ ]:
Norm = real(norm(ψ)); # Norm of the state 
Sx = real(expect(ψ, "Sx"))/Norm^2; # Magnetization along x
Sz = real(expect(ψ, "Sz"))/Norm^2; # Magnetization along z

In [ ]:
plot(Sx, xlabel = "j", ylabel = "Sx(j)")

# 3. TDVP for solution of Burgers’ equation (25 points)

### a) Solve Burgers’ equation using TDVP (22 points).
Consider a system with N = 8 qubits, spatial range 0 ≤ x < 1,
Re = 1000, time step Δt = 10−3, final time T = 0.24 and a Gaussian initial condition given by

In [ ]:
using HDF5
using ITensors
using LinearAlgebra
using Plots
using ITensorMPS
using Observers
using LaTeXStrings
plot_font = "Computer Modern";

In [ ]:
N = 8; # Number of sites
s_x = siteinds("Qubit", N); # Indices for all sites
xb = range(0, (1-(1/2^N)), length=2^N); # Sampling grid points

In [ ]:

step_size = 1/(2^N);
Re=1000;
dt=0.001;
final_t=0.24;
numsteps = Int.(round(final_t/dt)); # Number of time steps
χ = 8; # Maximum truncation parameter
cutoff = 1e-13; # Truncation error
tb = range(0, final_t, step=dt);
f(x) = exp(-8*pi*(x-0.5).^2);
yb = f.(xb);

plot(xb, yb)

In [ ]:
ϕ = MPS(yb, s_x, cutoff=cutoff, maxdim=χ);

orthogonalize!(ϕ,1)

@show linkdims(ϕ);


In [ ]:
# Left shift:

left_ls = zeros(2, 2, 2) 
middle_ls = zeros(2, 2, 2, 2) 
right_ls = zeros(2, 2, 2) 

left_ls[1, 2, 2] = 1
left_ls[2, 1, 1] = 1
middle_ls[1, 1, 1, 1] = 1
middle_ls[1, 2, 2, 1] = 1
middle_ls[2, 2, 1, 1] = 1
middle_ls[2, 1, 2, 2] = 1
right_ls[1, 1, 1] = 1
right_ls[1, 2, 2] = 1
right_ls[2, 1, 2] = 1
right_ls[2, 2, 1] = 1

H_ls = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_ls[i] = ITensor(left_ls, s_x[i], s_x[i]', α[i]);
    elseif i == N # last site
        H_ls[i] = ITensor(right_ls, α[i-1], s_x[i], s_x[i]');
    else # middle site
        H_ls[i] = ITensor(middle_ls, α[i-1], s_x[i], s_x[i]', α[i]);
    end
end

orthogonalize!(H_ls,1);

# Right shift:

left_rs = zeros(2, 2, 2) 
middle_rs = zeros(2, 2, 2, 2) 
right_rs = zeros(2, 2, 2) 

left_rs[1, 2, 1] = 1
left_rs[2, 1, 2] = 1
middle_rs[1, 1, 1, 1] = 1
middle_rs[1, 2, 2, 1] = 1
middle_rs[2, 1, 2, 1] = 1
middle_rs[2, 2, 1, 2] = 1
right_rs[1, 1, 1] = 1
right_rs[1, 2, 2] = 1
right_rs[2, 1, 2] = 1
right_rs[2, 2, 1] = 1

H_rs = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_rs[i] = ITensor(left_rs, s_x[i], s_x[i]', α[i]);
    elseif i == N # last site
        H_rs[i] = ITensor(right_rs, α[i-1], s_x[i], s_x[i]');
    else # middle site
        H_rs[i] = ITensor(middle_rs, α[i-1], s_x[i], s_x[i]', α[i]);
    end
end

orthogonalize!(H_rs,1);

# Identity:

left_id = zeros(2, 2, 1) 
middle_id = zeros(1, 2, 2, 1) 
right_id = zeros(1, 2, 2) 

left_id[1, 1, 1] = 1
left_id[2, 2, 1] = 1
middle_id[1, 1, 1, 1] = 1
middle_id[1, 2, 2, 1] = 1
right_id[1, 1, 1] = 1
right_id[1, 2, 2] = 1

H_id = MPO(N);

α = [Index(1) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_id[i] = ITensor(left_id, s_x[i], s_x[i]', α[i]);
    elseif i == N # last site
        H_id[i] = ITensor(right_id, α[i-1], s_x[i], s_x[i]');
    else # middle site
        H_id[i] = ITensor(middle_id, α[i-1], s_x[i], s_x[i]', α[i]);
    end
end

orthogonalize!(H_id,1);

# Define derivatives
#H_dudx = 0.5*(H_ls - H_rs)/step_size; # First derivative
#H_d2udx2 = (H_rs + H_ls - 2*H_id)/(step_size^2); # Second derivative


In [ ]:
#hh=deepcopy(H_rs);
#C=MPO(N);
#C = prime(hh)*H_rs*H_rs*H_rs
H_rs4=apply(H_rs,H_rs,H_rs,H_rs);
H_ls4=apply(H_ls,H_ls,H_ls,H_ls);
H_rs3=apply(H_rs,H_rs,H_rs);
H_ls3=apply(H_ls,H_ls,H_ls);
H_rs2=apply(H_rs,H_rs);
H_ls2=apply(H_ls,H_ls);

H_dudx= (1/280*H_rs4-4/105*H_rs3+1/5*H_rs2-4/5*H_rs+4/5*H_ls-1/5*H_ls2+4/105*H_ls3-1/280*H_ls4)/step_size;
H_d2udx2 = (-1/560*H_rs4+8/315*H_rs3-1/5*H_rs2+8/5*H_rs-205/72*H_id+8/5*H_ls-1/5*H_ls2+8/315*H_ls3-1/560*H_ls4)/(step_size^2);

In [ ]:
#create hadamard_product_MPO
had = MPO(N);
had .= convert(MPO, ϕ)
for i = 1:N
    had[i] *= δ(s_x[i], s_x[i]', s_x[i]'')
end

had = replaceprime(had, 2 => 0)

#ψ3_Had = apply(had, Deriv_x1)

#@show linkdims(ψ3_Had);

In [ ]:
function Advection_Diffusion_TDVP(ψ0, H_dudx, H_d2udx2, Re, s, N, dt, numsteps, χ, ϵ)
    
    ψ_list = []
    ψ = ψ0
    err_list = zeros(numsteps)
    
    
    # Perfor time evolution
    for t = 1:numsteps
        had = MPO(N);
        had .= convert(MPO, ψ)
        for i = 1:N
             had[i] *= δ(s[i], s[i]', s[i]'')
        end

        had = replaceprime(had, 2 => 0)

        # Define full MPO
        H_2=apply(had,H_dudx);
        H = (1/Re)*H_d2udx2 - H_2;    
        # Advance state by one time step using TDVP, and truncate afterwards
        ψ = tdvp(H, dt, ψ; time_step = dt, cutoff = ϵ, outputlevel=0, normalize=false)
        truncate!(ψ, maxdim = χ)  

        if t % (numsteps/4) == 0
            println("Calculating function for $(t*dt) s")
            Big_ψ = contract(ψ) # Recovering the original big tensor
            J_recon = Array(Big_ψ, s) # Changing the format from ITensor to Julia array for plotting
            ys_recon = reshape(J_recon,2^N,1) # Reshaping from N-dimentional tensor to regular array
        
            push!(ψ_list, ys_recon);
        end

    end

    return ψ_list, maxlinkdim(ψ);

end

In [ ]:
ψ_list, max_χ = Advection_Diffusion_TDVP(ϕ, H_dudx, H_d2udx2, Re, s_x, N, dt, numsteps, χ, cutoff);

### b) In a single figure, plot the results of u(x, t) as a function of x for truncation parameter χ = 8 and times t = 0, 0.06, 0.12, 0.18, 0.24. You can compare with the results of Homework 2 to verify the correctness of your implementation (3 points).

In [ ]:
plot(xb, yb, labels="t = 0", lw=1, linestyle=:solid, palette=:okabe_ito, framestyle=:box, dpi=1000, gridalpha = 0)
plot!(xb, ψ_list[1], labels="t = 0.06", lw=1, linestyle=:dash, xguidefontsize=15, yguidefontsize=15,legendfontsize=10)
plot!(xb, ψ_list[2], labels="t = 0.12", lw=1, linestyle=:dot)
plot!(xb, ψ_list[3], labels="t = 0.18", lw=1, linestyle=:dashdot, color = 6)
plot!(xb, ψ_list[4], labels="t = 0.24", lw=1, linestyle=:dashdotdot)


# 4. Fit Algorithm for solution of Burgers’ equation (15 points)

### a) Solve Burgers’ equation, with the same initial condition and parameters as in the previous exercise, using the DMRG-like Fit algorithm (12 points).

In [ ]:
function Advection_Diffusion_Fit(ψ0, H_dudx, H_d2udx2,H_id,Re, s, N, dt, numsteps, χ, ϵ)
    
    ψ_list = []
    ψ = ψ0
    nsweeps = 5
    init = randomMPS(s)

    
    
    # Perfor time evolution
    for t = 1:numsteps
        had = MPO(N);
        had .= convert(MPO, ψ)
        for i = 1:N
             had[i] *= δ(s[i], s[i]', s[i]'')
        end

        had = replaceprime(had, 2 => 0)

        # Define full MPO
        H_2=apply(had,H_dudx);
        H_old = (1/Re)*H_d2udx2 - H_2;    
        
        H = dt*H_old+ H_id;    
        
        # Advance state by one time step using Fit algorithm
        ψ = apply(H, ψ, alg="fit", init=init, maxdim=χ, cutoff=ϵ, nsweeps=nsweeps, outputlevel=0, normalize=false) 
        #truncate!(ψ, maxdim = χ)  

        if t % (numsteps/4) == 0
            println("Calculating function for $(t*dt) s")
            Big_ψ = contract(ψ) # Recovering the original big tensor
            J_recon = Array(Big_ψ, s) # Changing the format from ITensor to Julia array for plotting
            ys_recon = reshape(J_recon,2^N,1) # Reshaping from N-dimentional tensor to regular array
        
            push!(ψ_list, ys_recon);
        end

    end

    return ψ_list, maxlinkdim(ψ);

end

In [ ]:
ψ_list2, max_χ = Advection_Diffusion_Fit(ϕ, H_dudx, H_d2udx2, H_id,Re, s_x, N, dt, numsteps, χ, cutoff);

### b) In a single figure, plot the results of u(x, t) as a function of x for truncation parameter χ = 8 and times t = 0, 0.06, 0.12, 0.18, 0.24. You can compare with the results of Homework 2 and of the previous exercise to verify the correctness of your implementation (3 points).

In [ ]:
plot(xb, yb, labels="t = 0", lw=1, linestyle=:solid, palette=:okabe_ito, framestyle=:box, dpi=1000, gridalpha = 0)
plot!(xb, ψ_list2[1], labels="t = 0.06", lw=1, linestyle=:dash, xguidefontsize=15, yguidefontsize=15,legendfontsize=10)
plot!(xb, ψ_list2[2], labels="t = 0.12", lw=1, linestyle=:dot)
plot!(xb, ψ_list2[3], labels="t = 0.18", lw=1, linestyle=:dashdot, color = 6)
plot!(xb, ψ_list2[4], labels="t = 0.24", lw=1, linestyle=:dashdotdot)
